## LLM 調用

In [ ]:
import os

from langchain.embeddings.base import Embeddings
from langchain.messages import (AIMessageChunk, HumanMessage, SystemMessage,
                                ToolMessage)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import OllamaEmbeddings
from langchain_openai import ChatOpenAI
from langchain_postgres.vectorstores import PGVector
from openai import OpenAI
from pydantic import BaseModel, Field

from src.config.constant import (EMBEDDING_MODEL, OLLAMA_LOCAL, OLLAMA_URL,
                                 PG_COLLECTION, PROJECT_ROOT, SYSTEM_PROMPT)
from src.database import postgreSQL_conn as pgc

In [2]:
# 建立embedding類別
class LmStudioEmbeddings(Embeddings):
    def __init__(self, model_name, url):
        self.model_name = model_name
        self.url = url
        self.client = OpenAI(base_url=url, api_key="lm-studio")

    def embed_query(self, text: str):
        response = self.client.embeddings.create(input=text,model=self.model_name)
        return response.data[0].embedding

    def embed_documents(self, texts: list[str]):
        # 回傳多個文件的 embedding
        response = self.client.embeddings.create(input=texts,model=self.model_name)
        return [x.embedding for x in response.data]
        # return [self.model.encode(t).tolist() for t in texts]


class stream_chat_bot:
    def __init__(self, llm, tools):
        # 初始化對話機器人，傳入 LLM 與可用工具列表
        self.tools = tools
        # 將 LLM 綁定（bind）工具，使其具備自動呼叫工具的能力
        self.llm_with_tools = llm.bind_tools(tools)

        # 系統提示詞（System Prompt），用來設定 LLM 的角色與行為
        system_prompt = SYSTEM_PROMPT
        # 初始化訊息列表，第一條訊息是系統指令
        self.message = [SystemMessage(system_prompt)]

        # 將 LLM 的回應解析為純文字格式的工具
        self.str_parser = StrOutputParser()


    def chat_generator(self, text):
        """
        主對話生成函式（生成器形式）。
        逐步執行 LLM 回應與工具調用，並即時回傳每一步的結果。
        """
        # 將使用者的輸入加入訊息列表
        self.message.append(HumanMessage(text))

        while True:
            # 呼叫 LLM，傳入完整訊息歷史
            final_ai_message = AIMessageChunk(content="")
            for chunk in self.llm_with_tools.stream(self.message):
                final_ai_message += chunk
                if hasattr(chunk, 'content') and chunk.content:
                    yield self.str_parser.invoke(chunk)

            response = final_ai_message

            # 將 LLM 回應加入訊息列表
            self.message.append(response)

            # 檢查 LLM 是否要求呼叫工具
            is_tools_call = False
            for tool_call in response.tool_calls:
                is_tools_call = True

                # # 顯示 LLM 要執行的工具名稱與參數
                msg = f'[執行]: {tool_call["name"]}({tool_call["args"]})\n-----------\n' #完整訊息
                # # msg = f'[執行]: {tool_call["name"]}()\n\n' #簡易訊息
                yield msg  # 使用 yield 讓結果能即時顯示在輸出中

                # 實際執行工具（根據工具名稱動態呼叫對應物件）
                tool_result = globals()[tool_call['name']].invoke(tool_call['args']) 

                # # 顯示工具執行結果
                msg = f'[結果]: {tool_result}\n-----------\n'
                yield msg

                # 將工具執行結果封裝成 ToolMessage 回傳給 LLM
                tool_message = ToolMessage(
                    content=str(tool_result),          # 工具執行的文字結果
                    name=tool_call["name"],            # 工具名稱
                    tool_call_id=tool_call["id"],      # 工具呼叫 ID（讓 LLM 知道對應哪個呼叫）
                )
                # 將工具回傳結果加入訊息列表，提供 LLM 下一輪參考
                self.message.append(tool_message)

            # 若這一輪沒有任何工具呼叫，表示 LLM 已經生成最終回覆
            if is_tools_call == False:
                # 將 LLM 回應解析成純文字並輸出
                # yield self.str_parser.invoke(response)
                return  # 結束對話流程


    def chat(self, text, print_output=False):
        """
        封裝版對話函式。
        會收集 chat_generator 的所有輸出，並組合成完整的回覆字串。
        """
        msg = ''
        # 逐步取得 chat_generator 的產出內容
        for chunk in self.chat_generator(text):
            msg += f"{chunk}"
            if print_output:
                print(chunk, end='')
        # 回傳最終組合的對話內容
        return msg

In [ ]:
# 建立embedding連線
embeddings = OllamaEmbeddings(
    model=EMBEDDING_MODEL,
    base_url=OLLAMA_LOCAL
)

# 載入向量資料庫
pg_url = pgc.connect_to_pgSQL()
vector_store = PGVector(
        embeddings=embeddings,
        collection_name=PG_COLLECTION,
        connection=pg_url,
        use_jsonb=True,
    )

In [4]:
class GetQAInput(BaseModel):
    question: str = Field(description="查詢的問題文字")
    k: int = Field(default=5, description="要回傳的文件數量")

@tool(args_schema=GetQAInput)
def get_qa(question, n=10, k=2):
    """
    根據使用者提出的問題，從向量資料庫中檢索出最相關的 K 筆問答文件。

    Args:
        question (str): 查詢的問題文字。
        n (int): 搜尋子文件的數量。
        k (int): 要回傳的文件數量，預設為 2。

    Returns:
        documents: 檢索到的相似文件列表。
    """
    docs = vector_store.similarity_search(question, k=n)
    seen_ids = set()
    documents = []
    for doc in docs:
        if doc.metadata["parent_id"] not in seen_ids:
            seen_ids.add(doc.metadata["parent_id"])
            parent_docs = vector_store.similarity_search(query="",k=1,filter={"doc_id": doc.metadata["parent_id"]})
            if len(parent_docs) > 0:
                documents.append(parent_docs[0])

    return documents[:k]

In [5]:
# 串接Gemini
API_KEY = os.getenv("GOOGLE_API")
model_name = 'gemini-3-flash-preview'

llm = ChatGoogleGenerativeAI(
    model=model_name,
    google_api_key=API_KEY
)

In [6]:
# # 串接LM Studio
# model_name = 'gemma-3-12b-it'  # 指定模型名稱，模型名稱會根據下載的模型不同而改變

# base_url = 'http://192.168.0.109:1234/v1'  # LM Studio 本地伺服器的URL

# llm = ChatOpenAI(
#     model=model_name,
#     openai_api_key="not-needed",
#     openai_api_base=base_url
# )

In [7]:
bot = stream_chat_bot(llm, [get_qa])

In [ ]:
for x in bot.chat_generator("上古卷軸online的評價如何"):
    print(x, end='')

In [ ]:
for x in bot.chat_generator("請介紹上古卷軸online的遊戲內容"):
    print(x, end='')